<a href="https://colab.research.google.com/github/AbooMardiiyah/IndabaX-2022-3rd-place-solution/blob/main/deep_indaba_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importing the important libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder



In [3]:
train_data = pd.read_csv("/content/train.csv")
test_data = pd.read_csv("/content/test.csv")
train_data.head()

,surveyid,village,survey_date,femaleres,age,married,children,hhsize,edu,hh_children,...,given_mpesa,amount_given_mpesa,received_mpesa,amount_received_mpesa,net_mpesa,saved_mpesa,amount_saved_mpesa,early_survey,depressed,day_of_week
0,926,91,23-Nov-61,1,28.0,1,4,6,10,0,...,0,0.0,0,0.000000,0.000000,1,0.000000,0,0,5
1,747,57,24-Oct-61,1,23.0,1,3,5,8,0,...,0,0.0,1,4.804611,4.804611,0,0.000000,0,1,3
2,1190,115,05-Oct-61,1,22.0,1,3,5,9,0,...,0,0.0,0,8.007685,8.007685,1,0.000000,0,0,5
3,1065,97,23-Sep-61,1,27.0,1,2,4,10,2,...,0,0.0,0,0.000000,0.000000,1,1.249199,0,0,0
4,806,42,12-Sep-61,0,59.0,0,4,6,10,4,...,0,0.0,0,0.000000,0.000000,0,0.000000,0,0,3


## A little preprocessing

In [4]:
split = train_data.shape[0]
#combining the train and test data
train_test= pd.concat([train_data, test_data], axis=0)
train_test['age'] = train_test['age'].apply(lambda x: str(x) )
train_test['age'] = train_test['age'].apply(lambda x: str(0) if x == ".d" else x)
train_test['age'] = train_test['age'].apply(lambda x: float(x))

le = LabelEncoder()

train_test['survey_date'] = train_test['survey_date'].apply(lambda x: str(x))
le.fit(train_test['survey_date'])
train_test['survey_date'] = le.transform(train_test['survey_date'])

Null = train_test.isnull().sum()
Null = [keys for keys, values in Null.items() if values > 0]
for i in Null:
    train_test[i] = train_test[i].interpolate()

In [5]:
train_test.head(10)

,surveyid,village,survey_date,femaleres,age,married,children,hhsize,edu,hh_children,...,given_mpesa,amount_given_mpesa,received_mpesa,amount_received_mpesa,net_mpesa,saved_mpesa,amount_saved_mpesa,early_survey,depressed,day_of_week
0,926,91,144,1,28.0,1,4,6,10,0,...,0,0.0,0,0.000000,0.000000,1,0.000000,0,0.0,5
1,747,57,150,1,23.0,1,3,5,8,0,...,0,0.0,1,4.804611,4.804611,0,0.000000,0,1.0,3
2,1190,115,26,1,22.0,1,3,5,9,0,...,0,0.0,0,8.007685,8.007685,1,0.000000,0,0.0,5
3,1065,97,147,1,27.0,1,2,4,10,2,...,0,0.0,0,0.000000,0.000000,1,1.249199,0,0.0,0
4,806,42,78,0,59.0,0,4,6,10,4,...,0,0.0,0,0.000000,0.000000,0,0.000000,0,0.0,3
5,483,25,52,1,35.0,1,6,8,10,6,...,0,0.0,0,0.000000,0.000000,0,0.000000,0,0.0,6
6,849,130,190,0,34.0,0,1,3,9,1,...,0,0.0,0,0.000000,0.000000,0,0.000000,0,1.0,3
7,1386,72,15,1,21.0,1,2,4,10,2,...,0,0.0,0,0.000000,0.000000,0,0.000000,0,0.0,1
8,930,195,75,1,32.0,1,7,9,9,7,...,0,0.0,0,0.000000,0.000000,0,0.000000,0,0.0,1
9,390,33,172,1,29.0,1,4,6,10,0,...,0,0.0,0,0.000000,0.000000,0,0.000000,0,0.0,3


In [ ]:
# train_test.drop(columns=['surveyid','village','survey_date'],inplace=True)

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
 
# df_scaled= scaler.fit_transform(train_test.to_numpy())

In [ ]:
# train_test=pd.DataFrame(df_scaled,columns=train_test.columns)
# train_test.head()

In [6]:
train_df = train_test[:split]
test_df = train_test[split:]
train_df.dropna(inplace=True)
train_df.reset_index(drop=True, inplace=True)



/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


feature selection

In [ ]:

# from sklearn.ensemble import RandomForestClassifier

In [ ]:
# import pandas as pd
# from sklearn.feature_selection import SelectFromModel

In [7]:
y_train = train_df['depressed']
x_train = train_df.drop(labels=['depressed'], axis=1)
x_test = test_df.drop(labels=['depressed'], axis=1)

In [ ]:
# sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))
# sel.fit(x_train, y_train)

In [ ]:
# sel.get_support()

In [ ]:
# selected_feat= x_train.columns[(sel.get_support())]
# len(selected_feat)

In [ ]:
# print(selected_feat)

In [ ]:
# feat=selected_feat.tolist()


In [ ]:
# pd.Series(sel.estimator_.feature_importances_.ravel()).hist()

In [ ]:
# train.head()

In [ ]:
# train=train[feat]

In [ ]:
# train.head()

In [ ]:
# x_train = train[feat]
# x_test = test[feat]

In [ ]:

clf = AdaBoostClassifier(n_estimators=90,learning_rate=0.01)



In [8]:

# # n_jobs=-1 to allow run it on all cores
# params = {
#     'learning_rate': [0.01,0.05,0.1],
#     'max_depth':range(3,12,3),
#     'min_samples_split'=[100,200],
# }

# gs2 = GridSearchCV(xgb.XGBClassifier(n_jobs=-1), params, n_jobs=-1, cv=KFold(n_splits=3), scoring='roc_auc')
# gs2.fit(x_train, y_train)


In [9]:

# print('Best score:', gs2.best_score_)
# print('Best score:', gs2.best_params_)

In [10]:
# param_test3 = {'random_state':range(2,15,2),'max_depth':range(3,12,3)}
# gsearch3 = GridSearchCV(estimator = GradientBoostingClassifier( n_estimators=90), 
# param_grid = param_test3, scoring='roc_auc',n_jobs=4, cv=5)
# gsearch3.fit(x_train,y_train)
# gsearch3.best_params_, gsearch3.best_score_

In [11]:
# params = {
#     'n_estimators':range(10,50,10),
#     'random_state':range(3,12,3)
# }

# gs1 = GridSearchCV(RandomForestClassifier(n_jobs=-1), params, n_jobs=-1, cv=KFold(n_splits=3), scoring='roc_auc')
# gs1.fit(x_train, y_train)

# print('Best score:', gs1.best_score_)
# print('Best score:', gs1.best_params_)

##Stacking the models with their best parameters

In [ ]:
# !pip install mlxtend
# import six
# import sys
# sys.modules['sklearn.externals.six'] = six

In [ ]:
# import mlxtend
# from mlxtend.classifier import StackingClassifier
# from sklearn.linear_model import LogisticRegression

In [ ]:
# model1 = RandomForestClassifier(random_state=3, n_estimators=20)
# model2= xgb.XGBClassifier(seed=3)
# model3 = GradientBoostingClassifier(n_estimators=120, max_depth=3, random_state=8) 

# stack = StackingClassifier(classifiers=[model1, model2, model3], meta_classifier=clf)
# stack.fit(x_train, y_train)
    
#     # make class predictions for the testing set
# y_pred_class = stack.predict(x_test)
    

In [ ]:
# sub = pd.DataFrame({"surveyid": df_test['surveyid'],  "depressed": y_pred_class})
# sub= subm[['surveyid', 'depressed']]
# sub.to_csv("test.csv", index = False)

##Blending Gradientbosting, xgb and randomforest with their hyperparameters

In [12]:
# model = GradientBoostingClassifier(n_estimators=90) 
model1= GradientBoostingClassifier(n_estimators=90, max_depth=3, random_state=8)
model1.fit(x_train,y_train)
gb = model1.predict(x_test)

model2= xgb.XGBClassifier(seed=3,max_depth=4,min_samples_split=100,learning_rate=0.1)
model2.fit(x_train, y_train)
xgb = model2.predict_proba(x_test)
        
model3 = RandomForestClassifier(random_state=3,n_estimators=20)
model3.fit(x_train, y_train)
rf = model3.predict(x_test)


xgb_pred = []
for p in xgb:
    if 0.49 < p[1] < 0.6:
        xgb_pred.append(1)
    else:
        xgb_pred.append(0)



blend = []
for p in range(len(gb)):
    if (gb[p] > 0) | (xgb_pred[p] > 0) | (rf[p] > 0):
        blend.append(1)
    else:
        blend.append(0)


In [13]:
for i in blend:
  if i==1:
    print(i)

1
1
1
1
1
1
1
1
1
1


In [14]:
sub = pd.DataFrame({"surveyid": test_df['surveyid'],  "depressed": blend})
sub = sub[['surveyid', 'depressed']]
sub.to_csv("sub_blend.csv", index = False)

In [ ]:

# sub = pd.DataFrame({"surveyid": df_test['surveyid'],  "depressed": gb_pred})
# sub = sub[['surveyid', 'depressed']]
# sub.to_csv("sub_gb.csv", index = False)

In [ ]:
# sub = pd.DataFrame({"surveyid": df_test['surveyid'],  "depressed": xgb_pred})
# sub = sub[['surveyid', 'depressed']]
# sub.to_csv("sub_xgb.csv", index = False)

In [ ]:
# sub = pd.DataFrame({"surveyid": df_test['surveyid'],  "depressed": rf_pred})
# sub = sub[['surveyid', 'depressed']]
# sub.to_csv("sub_rf.csv", index = False)